In [1]:
import numpy as np
import scipy as sp

import scipy
from scipy.stats import multivariate_normal
from scipy       import optimize

import skimage

import matplotlib.pyplot as plt

import pylab

from glob import glob
import sys
import os

from tqdm import tqdm

import math

from glob import glob

import matplotlib.pyplot as plt

import itk
from itk import TubeTK as ttk

import av
import ffmpeg

import json

import site
site.addsitedir('../../ARGUS')
from ARGUSUtils_IO import *
from ARGUSUtils_Linearization import *

In [2]:
def insert_overlays_into_volume(pathname, pluClass):
    data_overlay_paths = sorted(glob(pathname+"-Annotations-Linear/*.overlay.mha"))
    data_video_dir = pathname+"-Linear/"
    
    num_overlays = len(data_overlay_paths)
    
    i=0
    (next_path,next_filename) = os.path.split(data_overlay_paths[i])
    next_video_name = data_video_dir+next_filename.split('.')[0]+'.mha'
    while i<num_overlays:
        print(next_video_name)
        video_name = next_video_name
        filename = next_filename
        vidimg = itk.imread(video_name, itk.F)
        nbs = 0
        while np.mean(vidimg[nbs])<=1:
            nbs += 1
        print("   Video Blank: 0 -", nbs)
        for j in range(0,nbs):
            vidimg[j] = vidimg[nbs]
        nbs = vidimg.shape[0]-1
        while np.mean(vidimg[nbs])<=1:
            nbs -= 1
        print("   Video Blank: ", nbs+1, "-", vidimg.shape[0])
        for j in range(nbs+1,vidimg.shape[0]):
            vidimg[j] = vidimg[nbs]
        new_video_name = pathname+"-Annotations-Linear/"+filename.split('.')[0]+'.mha'
        itk.imwrite(vidimg, new_video_name)
        
        vidimg.FillBuffer(0)
        print("Processing", video_name)
        while next_video_name==video_name and i<num_overlays:
            filename = next_filename
            frame_num = int(filename.split('.')[1].split('-')[2][1:6])
            print("   ", frame_num)
            ovimg = itk.imread(data_overlay_paths[i], itk.UC)
            vidimg[frame_num,:,:] = ovimg
            i += 1
            if i<num_overlays:
                (next_path,next_filename) = os.path.split(data_overlay_paths[i])
                next_video_name = data_video_dir+next_filename.split('.')[0]+'.mha'
        new_video_name = pathname+"-Annotations-Linear/"+filename.split('.')[0]+'.inserted-overlay.nii.gz'
        print("Writing", new_video_name)
        itk.imwrite(vidimg, new_video_name, compression=True)
        
        vid = itk.GetArrayFromImage(vidimg)
        
        Interp = itk.MorphologicalContourInterpolator.New(itk.GetImageFromArray(vid.astype(np.ubyte)))
        Interp.Update()
        vidimg = Interp.GetOutput()
        vid = itk.GetArrayFromImage(vidimg).astype(np.ubyte)
        
        label = vid
        
        rib = np.where(vid==2, 1, 0)
        ribimg = itk.GetImageFromArray(rib.astype(np.ubyte))
        ConnComp = ttk.SegmentConnectedComponents.New(ribimg)
        ConnComp.Update()
        ribimg = ConnComp.GetOutput()
        rib = itk.GetArrayFromImage(ribimg)
        maxCC = rib.max()
        print("maxCC(rib) =", maxCC)
        for cc in range(1,maxCC+1):
            print("CC(rib) =", cc)
            ribCC = np.where(rib==cc,2,0)
            nbs = 0
            while nbs<ribCC.shape[0] and np.sum(ribCC[nbs])==0:
                nbs += 1
            if nbs<ribCC.shape[0]:
                print("   Blank: 0 -", nbs)
                for j in range(0,nbs):
                    ribCC[j] = ribCC[nbs]
                nbs = ribCC.shape[0]-1
                while np.sum(ribCC[nbs])==0:
                    nbs -= 1
                print("   Blank: ", nbs-1, "-", ribCC.shape[0])
                for j in range(nbs+1,ribCC.shape[0]):
                    ribCC[j] = ribCC[nbs]
                label = np.maximum(label, ribCC.astype(np.ubyte), dtype=np.ubyte)
        
        plu = np.where(vid==1, vid, 0)
        pluimg = itk.GetImageFromArray(plu.astype(np.ubyte))
        ConnComp = ttk.SegmentConnectedComponents.New(pluimg)
        ConnComp.Update()
        pluimg = ConnComp.GetOutput()
        plu = itk.GetArrayFromImage(pluimg)
        maxCC = plu.max()
        print("maxCC(plu) =", maxCC)
        for cc in range(1,maxCC+1):
            print("CC(plu) =", cc)
            pluCC = np.where(plu==cc,pluClass,0)
            nbs = 0
            while nbs<pluCC.shape[0] and np.sum(pluCC[nbs])==0:
                nbs += 1
            if nbs<pluCC.shape[0]:
                print("   Blank: 0 -", nbs)
                for j in range(0,nbs):
                    pluCC[j] = pluCC[nbs]
                nbs = pluCC.shape[0]-1
                while np.sum(pluCC[nbs])==0:
                    nbs -= 1
                print("   Blank: ", nbs-1, "-", pluCC.shape[0])
                for j in range(nbs+1,pluCC.shape[0]):
                    pluCC[j] = pluCC[nbs]
                label = np.maximum(label,pluCC.astype(np.ubyte),dtype=np.ubyte)
        vid = itk.GetImageFromArray(label)
        new_video_name = pathname+"-Annotations-Linear/"+filename.split('.')[0]+'.interpolated-overlay.nii.gz'
        print("Writing", new_video_name)
        itk.imwrite(vid, new_video_name, compression=True)

In [3]:
pathname = "../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding"
insert_overlays_into_volume(pathname, 1)
pathname = "../../Data/TrainingData/AR-UNet/BAMC-PTXSliding"
insert_overlays_into_volume(pathname, 3)

../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Linear/Image_262499828648_clean.mha
   Video Blank: 0 - 0
   Video Blank:  68 - 68
Processing ../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Linear/Image_262499828648_clean.mha
    0
    4
    6
    11
    17
    24
    45
    48
    56
    63
Writing ../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations-Linear/Image_262499828648_clean.inserted-overlay.nii.gz
maxCC(rib) = 5
CC(rib) = 1
CC(rib) = 2
   Blank: 0 - 0
   Blank:  62 - 68
CC(rib) = 3
   Blank: 0 - 0
   Blank:  62 - 68
CC(rib) = 4
   Blank: 0 - 0
   Blank:  4 - 68
CC(rib) = 5
   Blank: 0 - 12
   Blank:  62 - 68
maxCC(plu) = 3
CC(plu) = 1
CC(plu) = 2
   Blank: 0 - 0
   Blank:  62 - 68
CC(plu) = 3
   Blank: 0 - 0
   Blank:  62 - 68
Writing ../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations-Linear/Image_262499828648_clean.interpolated-overlay.nii.gz
../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Linear/image_1083297968960_clean.mha
   Video Blank: 0 - 0
  

maxCC(rib) = 3
CC(rib) = 1
CC(rib) = 2
   Blank: 0 - 0
   Blank:  39 - 61
CC(rib) = 3
   Blank: 0 - 0
   Blank:  39 - 61
maxCC(plu) = 2
CC(plu) = 1
CC(plu) = 2
   Blank: 0 - 0
   Blank:  39 - 61
Writing ../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations-Linear/image_2454526567135_CLEAN.interpolated-overlay.nii.gz
../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Linear/image_267456908021_clean.mha
   Video Blank: 0 - 0
   Video Blank:  61 - 61
Processing ../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Linear/image_267456908021_clean.mha
    0
    6
    20
    30
    33
    36
    39
    40
    49
    60
Writing ../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations-Linear/image_267456908021_clean.inserted-overlay.nii.gz
maxCC(rib) = 5
CC(rib) = 1
CC(rib) = 2
   Blank: 0 - 0
   Blank:  59 - 61
CC(rib) = 3
   Blank: 0 - 0
   Blank:  59 - 61
CC(rib) = 4
   Blank: 0 - 0
   Blank:  59 - 61
CC(rib) = 5
   Blank: 0 - 50
   Blank:  59 - 61
maxCC(plu) = 5
CC(plu) = 1
CC(plu)

maxCC(rib) = 3
CC(rib) = 1
CC(rib) = 2
   Blank: 0 - 0
   Blank:  59 - 61
CC(rib) = 3
   Blank: 0 - 0
   Blank:  59 - 61
maxCC(plu) = 2
CC(plu) = 1
CC(plu) = 2
   Blank: 0 - 0
   Blank:  59 - 61
Writing ../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations-Linear/image_603665940081_clean.interpolated-overlay.nii.gz
../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Linear/image_6056976176281_CLEAN.mha
   Video Blank: 0 - 0
   Video Blank:  60 - 60
Processing ../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Linear/image_6056976176281_CLEAN.mha
    0
    9
    16
    20
    36
    39
    40
    51
    54
    57
Writing ../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations-Linear/image_6056976176281_CLEAN.inserted-overlay.nii.gz
maxCC(rib) = 3
CC(rib) = 1
CC(rib) = 2
   Blank: 0 - 0
   Blank:  56 - 60
CC(rib) = 3
   Blank: 0 - 0
   Blank:  56 - 60
maxCC(plu) = 3
CC(plu) = 1
CC(plu) = 2
   Blank: 0 - 0
   Blank:  56 - 60
CC(plu) = 3
   Blank: 0 - 15
   Blank:  14 - 60
Writi

maxCC(rib) = 4
CC(rib) = 1
CC(rib) = 2
   Blank: 0 - 0
   Blank:  59 - 61
CC(rib) = 3
   Blank: 0 - 0
   Blank:  52 - 61
CC(rib) = 4
   Blank: 0 - 55
   Blank:  59 - 61
maxCC(plu) = 2
CC(plu) = 1
CC(plu) = 2
   Blank: 0 - 0
   Blank:  59 - 61
Writing ../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations-Linear/image_104548309385533_CLEAN.interpolated-overlay.nii.gz
../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Linear/image_104932526155699_CLEAN.mha
   Video Blank: 0 - 0
   Video Blank:  61 - 61
Processing ../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Linear/image_104932526155699_CLEAN.mha
    0
    6
    8
    10
    16
    20
    35
    40
    54
    60
Writing ../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations-Linear/image_104932526155699_CLEAN.inserted-overlay.nii.gz
maxCC(rib) = 3
CC(rib) = 1
CC(rib) = 2
   Blank: 0 - 0
   Blank:  59 - 61
CC(rib) = 3
   Blank: 0 - 11
   Blank:  59 - 61
maxCC(plu) = 2
CC(plu) = 1
CC(plu) = 2
   Blank: 0 - 0
   Blank:  59 - 61
Writ

   Blank: 0 - 1
   Blank:  53 - 60
Writing ../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations-Linear/image_1499268364374_clean.interpolated-overlay.nii.gz
../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Linear/image_1511338287338_clean.mha
   Video Blank: 0 - 0
   Video Blank:  61 - 61
Processing ../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Linear/image_1511338287338_clean.mha
    0
    7
    9
    13
    20
    30
    32
    36
    40
Writing ../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations-Linear/image_1511338287338_clean.inserted-overlay.nii.gz
maxCC(rib) = 4
CC(rib) = 1
CC(rib) = 2
   Blank: 0 - 0
   Blank:  39 - 61
CC(rib) = 3
   Blank: 0 - 0
   Blank:  39 - 61
CC(rib) = 4
   Blank: 0 - 8
   Blank:  11 - 61
maxCC(plu) = 4
CC(plu) = 1
CC(plu) = 2
   Blank: 0 - 0
   Blank:  39 - 61
CC(plu) = 3
   Blank: 0 - 0
   Blank:  39 - 61
CC(plu) = 4
   Blank: 0 - 0
   Blank:  39 - 61
Writing ../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations-Linear/image_151133

Processing ../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Linear/image_3384882513134_clean.mha
    0
    6
    8
    10
    20
    27
    29
    31
    34
    40
Writing ../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations-Linear/image_3384882513134_clean.inserted-overlay.nii.gz
maxCC(rib) = 3
CC(rib) = 1
CC(rib) = 2
   Blank: 0 - 0
   Blank:  39 - 60
CC(rib) = 3
   Blank: 0 - 0
   Blank:  39 - 60
maxCC(plu) = 3
CC(plu) = 1
CC(plu) = 2
   Blank: 0 - 0
   Blank:  39 - 60
CC(plu) = 3
   Blank: 0 - 0
   Blank:  39 - 60
Writing ../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations-Linear/image_3384882513134_clean.interpolated-overlay.nii.gz
../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Linear/image_3401832241774_clean.mha
   Video Blank: 0 - 0
   Video Blank:  61 - 61
Processing ../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Linear/image_3401832241774_clean.mha
    0
    6
    14
    19
    20
    34
    39
    40
    51
    60
Writing ../../Data/TrainingData/AR-UNet/BAMC-

In [4]:
pathname = "../../Data/TestingData/BAMC-PTXNoSliding"
insert_overlays_into_volume(pathname, 1)
pathname = "../../Data/TestingData/BAMC-PTXSliding"
insert_overlays_into_volume(pathname, 3)

../../Data/TestingData/BAMC-PTXNoSliding-Linear/image_417221672548_CLEAN.mha
   Video Blank: 0 - 0
   Video Blank:  61 - 61
Processing ../../Data/TestingData/BAMC-PTXNoSliding-Linear/image_417221672548_CLEAN.mha
    0
    5
    10
    17
    20
    26
    33
    40
    57
    60
Writing ../../Data/TestingData/BAMC-PTXNoSliding-Annotations-Linear/image_417221672548_CLEAN.inserted-overlay.nii.gz
maxCC(rib) = 3
CC(rib) = 1
CC(rib) = 2
   Blank: 0 - 0
   Blank:  59 - 61
CC(rib) = 3
   Blank: 0 - 0
   Blank:  59 - 61
maxCC(plu) = 2
CC(plu) = 1
CC(plu) = 2
   Blank: 0 - 0
   Blank:  59 - 61
Writing ../../Data/TestingData/BAMC-PTXNoSliding-Annotations-Linear/image_417221672548_CLEAN.interpolated-overlay.nii.gz
../../Data/TestingData/BAMC-PTXNoSliding-Linear/image_426794579576_CLEAN.mha
   Video Blank: 0 - 0
   Video Blank:  60 - 60
Processing ../../Data/TestingData/BAMC-PTXNoSliding-Linear/image_426794579576_CLEAN.mha
    0
    4
    7
    11
    14
    19
    20
    33
    37
    40
Writing 

In [3]:
? itk.imwrite